# ***Download*** the concatenated synthetic profile of a selected BGC-Argo float
e.g., https://data-argo.ifremer.fr/dac/coriolis/6903574/6903574_Sprof.nc

### <span style="color:red">--- User input begins ---</span>
Specify the WMOID of the BGC-Argo float you want to download

In [ ]:
wmoid = 6903574

### <span style="color:red">--- User input ends ---</span>

### Download

In [ ]:
# Loading necessary modules
import pandas as pd
import os
import urllib.request
import hashlib

# Making the directory for the selected float, if it does not exist.
os.makedirs(str(wmoid), exist_ok=True)

# Sometimes either the French or the US server is down. So try both HTTPS 
# (commenting out FTP because they are not accessible from the Notebook)
urls_to_try = [
    'https://data-argo.ifremer.fr/',
    'https://usgodae.org/pub/outgoing/argo/',
    #'ftp://ftp.ifremer.fr/ifremer/argo/',
    #'ftp://usgodae.org/pub/outgoing/argo/'
]

for i in range(len(urls_to_try)):
    try:
        ds = pd.read_csv(urls_to_try[i]+"argo_synthetic-profile_index.txt",skiprows=8)
        print("Successfully read the index file from "+urls_to_try[i])
        orgname = ds[ds['file'].str.contains(str(wmoid))]['file'].values[0].split('/')[0]
        url = urls_to_try[i]+"dac/"+orgname+"/"+str(wmoid)+"/"+str(wmoid)+"_Sprof.nc"
        try:
            urllib.request.urlretrieve(url,str(wmoid)+"/"+url.split('/')[-1])
            print("Download successful: "+str(wmoid)+"/"+url.split("/")[-1])
            break
        except:
            print("Download unsuccessful")
    except:
        print(urls_to_try[i]+" is not working...")
else:
    print("Both servers seem to be down... try again later")